In [2]:
!pip install cuml
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from cuml.cluster import GaussianMixture
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
import joblib

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cuml
  Running setup.py clean for cuml
Failed to build cuml
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cuml)


ImportError: cannot import name 'GaussianMixture' from 'cuml.cluster' (/usr/local/lib/python3.12/dist-packages/cuml/cluster/__init__.py)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_path = "/content/drive/My Drive/Datasets/ML Project/data/recommended/training/training_cleaned.csv"
df = pd.read_csv(train_path)[[
        'proto', 'sport', 'dport',
        'min', 'max', 'mean', 'stddev',
        'state_number',
        'N_IN_Conn_P_SrcIP', 'N_IN_Conn_P_DstIP',
        'srate', 'drate'
    ]].dropna()
df.head()

In [ ]:
def dropna(X):
  return X.dropna()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
preprocessing_pipeline = Pipeline([
    ('encode_proto', OrdinalEncoder(handle_unknown='use_encoded_value', unkown_value=-1)),
    ('scaler', StandardScaler())
])

full_pipeline = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('gmm', GaussianMixture(
        n_components=19,
        covariance_type='full',
        reg_covar=1e-4,
        random_state=42,
        init_params='k-means++',
        n_init=10
    ))
])

cluster_labels = full_pipeline.fit_predict(df)

print(f"Unique cluster labels: {np.unique(cluster_labels)}")


In [ ]:
save_path = "/content/drive/My Drive/Datasets/ML Project/models/without/gmm.joblib"
joblib.dump(full_pipeline, save_path)